In [1]:
import subprocess
import sys
import time
from itertools import zip_longest
import json,datetime,schedule

from trader import write_signal
from funcs import get_data,get_entrys_exits,get_entry_signals, validate_dfmpl

In [2]:
TZOFFSET=5*60*60 

## input arguments
param_choice = 15
#if len(sys.argv)>1:
#    param_choice = int(sys.argv[1])

trade_params = json.load(open("trade_params.json","r"))
trade_param = trade_params["ver2"]["params"][param_choice]
thres_diff = float(trade_param["thres_diff"])
tickerpair = trade_param["tickerpair"]
interval = trade_param["interval"]
sl = float(trade_param["sl"])
tp = float(trade_param["tp"])
percentile = float(trade_param["percentile"])

In [3]:
%%time
#dfmpl2 = get_data(tickerpair,interval,limit=100,type="live")
import pickle
# with open("test_dfmpldata.pkl","wb") as f:
#     pickle.dump(dfmpl2,f)
with open("test_dfmpldata.pkl","rb") as f:
    dfmpl2 = pickle.load(f)

CPU times: total: 0 ns
Wall time: 996 µs


In [67]:
for i in range(55,58):
    print(i)
    dfmpl=dfmpl2.iloc[:i]
    #dfmpl
    entrys,exits,_,_,_,_,_,_,thres_diff = get_entrys_exits(dfmpl,percentile,thres_diff)
    #print(len(entrys),len(exits),entrys,exits)
    print(entrys+1)
    print(exits+1)

55
[19 22]
[21 23]
56
[19 22]
[21 23]
57
[19 22 56]
[21 23]


In [69]:
dfmpl.iloc[-5:]

,Open,High,Low,Close,Volume
Date,,,,,
2023-07-16 11:40:00,27.85,27.90,27.77,27.80,15508.20
2023-07-16 11:45:00,27.80,27.86,27.79,27.82,8530.92
2023-07-16 11:50:00,27.81,27.83,27.74,27.77,11226.53
2023-07-16 11:55:00,27.76,27.86,27.75,27.85,12788.93
2023-07-16 12:00:00,27.85,28.18,27.85,28.08,85446.23


In [6]:
#get all entry signal
entry_signals = get_entry_signals(entrys,dfmpl,onlybuy=True) 
new_entry=False
entry_df= None;buy=None
# todo , this will still report pervious entries as new entry on start
# todo redo run trades (possibly fixed)
for entry,exit in zip_longest(entrys,exits,fillvalue=None):
    if exit is None: # missing exit signal, so its a new enter signal
        buy_list = [ buy for ( entry_, buy, _ ) in entry_signals if entry_==entry]
        if len(buy_list)==1: 
            new_entry=True
            entry_df=dfmpl.iloc[entry]
            buy=buy_list[0]